# 最基本的手写数字识别

## 需要的包

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

## 是否使用cuda

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

使用设备: cuda


## 加载数据

In [5]:
# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 加载训练集和测试集
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

## 定义模型、损失函数和优化器

In [6]:
# 定义模型
model = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

print('定义模型 .. OK')

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

定义模型 .. OK


## 基于cpu的训练

In [7]:
%%time

# 训练模型
for epoch in range(5):  # 进行5个epoch的训练
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs.view(inputs.size(0), -1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # 每200个batch打印一次损失值
            print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('训练 .. OK')

Epoch: 1, Batch: 200, Loss: 1.551
Epoch: 1, Batch: 400, Loss: 0.731
Epoch: 1, Batch: 600, Loss: 0.529
Epoch: 1, Batch: 800, Loss: 0.456
Epoch: 2, Batch: 200, Loss: 0.403
Epoch: 2, Batch: 400, Loss: 0.371
Epoch: 2, Batch: 600, Loss: 0.361
Epoch: 2, Batch: 800, Loss: 0.349
Epoch: 3, Batch: 200, Loss: 0.344
Epoch: 3, Batch: 400, Loss: 0.325
Epoch: 3, Batch: 600, Loss: 0.320
Epoch: 3, Batch: 800, Loss: 0.313
Epoch: 4, Batch: 200, Loss: 0.307
Epoch: 4, Batch: 400, Loss: 0.310
Epoch: 4, Batch: 600, Loss: 0.288
Epoch: 4, Batch: 800, Loss: 0.290
Epoch: 5, Batch: 200, Loss: 0.281
Epoch: 5, Batch: 400, Loss: 0.274
Epoch: 5, Batch: 600, Loss: 0.281
Epoch: 5, Batch: 800, Loss: 0.276
训练 .. OK
CPU times: user 1min, sys: 1.4 s, total: 1min 1s
Wall time: 52.2 s


## 基于gpu的训练

设置 gpu

In [8]:
# 将模型移动到 GPU 上
model = model.to(device)

使用 gpu 训练模型

In [12]:
%%time
# 在训练循环开始前将数据移动到 GPU 上
for epoch in range(5):  # 进行5个epoch的训练
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # 将数据移动到 GPU 上

        optimizer.zero_grad()

        outputs = model(inputs.view(inputs.size(0), -1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # 每200个batch打印一次损失值
            print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('训练 .. OK')

Epoch: 1, Batch: 200, Loss: 0.189
Epoch: 1, Batch: 400, Loss: 0.180
Epoch: 1, Batch: 600, Loss: 0.187
Epoch: 1, Batch: 800, Loss: 0.184
Epoch: 2, Batch: 200, Loss: 0.176
Epoch: 2, Batch: 400, Loss: 0.175
Epoch: 2, Batch: 600, Loss: 0.170
Epoch: 2, Batch: 800, Loss: 0.178
Epoch: 3, Batch: 200, Loss: 0.169
Epoch: 3, Batch: 400, Loss: 0.168
Epoch: 3, Batch: 600, Loss: 0.168
Epoch: 3, Batch: 800, Loss: 0.167
Epoch: 4, Batch: 200, Loss: 0.151
Epoch: 4, Batch: 400, Loss: 0.161
Epoch: 4, Batch: 600, Loss: 0.159
Epoch: 4, Batch: 800, Loss: 0.159
Epoch: 5, Batch: 200, Loss: 0.150
Epoch: 5, Batch: 400, Loss: 0.155
Epoch: 5, Batch: 600, Loss: 0.142
Epoch: 5, Batch: 800, Loss: 0.151
训练 .. OK
CPU times: user 50.2 s, sys: 1.33 s, total: 51.5 s
Wall time: 51.7 s


## 基于gpu推理测试

In [11]:
# 在测试集上进行推理
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # 将数据移动到 GPU 上
        outputs = model(images.view(images.size(0), -1))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'测试模型的准确度: {(100 * correct / total):.2f}%')

测试模型的准确度: 94.43%
